In [109]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
# pip install mord
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
#Import math Library
import math

In [110]:
DATA = Path('C:\\Users\\tanve\\Documents\\206\\dmba\\')

### 1. Financial Condition of Banks. The file Banks.csv includes data on a sample of 20 banks. The “Financial Condition” column records the judgment of an expert on the financial condition of each bank. This outcome variable takes one of two possible values—weak or strong—according to the financial condition of the bank. The predictors are two ratios used in the financial analysis of banks: TotLns&Lses/Assets is the ratio of total loans and leases to total assets and TotExp/Assets is the ratio of total expenses to total assets. The target is to use the two ratios for classifying the financial condition of a new bank. Run a logistic regression model (on the entire dataset) that models the status of a bank as a function of the two financial measures provided. Specify the success class as weak (this is similar to creating a dummy that is 1 for financially weak banks and 0 otherwise), and use the default cutoff value of 0.5.

In [111]:
bank_df = pd.read_csv(DATA / 'Banks.csv')
print(bank_df.shape)
bank_df.head(2)

(20, 5)


,Obs,Financial Condition,TotCap/Assets,TotExp/Assets,TotLns&Lses/Assets
0,1,1,9.7,0.12,0.65
1,2,1,1.0,0.11,0.62


#### a. Write the estimated equation that associates the financial condition of a bank with its two predictors in three formats:

##### i. The logit as a function of the predictors

In [112]:
# logistic regression using Income only
predictors = ['TotLns&Lses/Assets', 'TotExp/Assets']
outcome = 'Financial Condition'
y = bank_df[outcome]
X = bank_df[predictors]


In [113]:
logit_reg_predictor = LogisticRegression(penalty="l2", C=1e42, solver='liblinear', tol=1e-8)
logit_reg_predictor.fit(X, y)

print('intercept ', logit_reg_predictor.intercept_[0])
print(pd.DataFrame({'coefficient': logit_reg_predictor.coef_[0]}, index=X.columns).transpose())
print()
#print('AIC', AIC_score(valid_y, logit_reg_predictor.predict(valid_X), df = len(train_X.columns) + 1))

intercept  -14.721008090957572
             TotLns&Lses/Assets  TotExp/Assets
coefficient             8.37132      89.833916



##### ii. The odds as a function of the predictors

In [114]:
math.exp(-14.72100809095757+8.37132*1+89.833916*1)/math.exp(-14.72100809095757+8.37132*0+89.833916*0)

4.466754553491884e+42

##### iii. The probability as a function of the predictors

In [115]:
-14.72100809095757+8.37132*1+89.833916*1

83.48422790904243

#### b. Consider a new bank whose total loans and leases/assets ratio = 0.6 and total expenses/assets ratio = 0.11. From your logistic regression model, estimate the following four quantities for this bank (use Python to do all the intermediate calculations; show your final answers to four decimal places): the logit, the odds, the probability of being financially weak, and the classification of the bank (use cutoff = 0.5).

In [116]:
print('intercept ', round(logit_reg_predictor.intercept_[0],4))
print(pd.DataFrame({'coefficient': logit_reg_predictor.coef_[0].round(4)}, index=X.columns).transpose())
print()

intercept  -14.721
             TotLns&Lses/Assets  TotExp/Assets
coefficient              8.3713        89.8339



In [117]:
#odds
round(math.exp(-14.72100809095757+8.37132*0.6+89.833916*0.11)/math.exp(-14.72100809095757+8.37132*0+89.833916*0), 4)

2971337.5413

In [118]:
#log(odds) Logit
round(-14.72100809095757+8.37132*0.6+89.833916*0.11, 4)

0.1835

In [119]:
data = {'TotLns&Lses/Assets': [0.6], 'TotExp/Assets': [0.11]}
test_df = pd.DataFrame(data)
predictions = logit_reg_predictor.predict(test_df)

In [120]:
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
predictions_nominal

[1]

#### c. The cutoff value of 0.5 is used in conjunction with the probability of being financially weak. Compute the threshold that should be used if we want to make a classification based on the odds of being financially weak, and the threshold for the corresponding logit.

In [149]:
for i in np.linspace(0.0, 0.5, 51):
    print(i)
    print(str(round(-14.72100809095757+8.37132*i+89.833916*i, 4)))

0.0
-14.721
0.01
-13.739
0.02
-12.7569
0.03
-11.7749
0.04
-10.7928
0.05
-9.8107
0.06
-8.8287
0.07
-7.8466
0.08
-6.8646
0.09
-5.8825
0.1
-4.9005
0.11
-3.9184
0.12
-2.9364
0.13
-1.9543
0.14
-0.9723
0.15
0.0098
0.16
0.9918
0.17
1.9739
0.18
2.9559
0.19
3.938
0.2
4.92
0.21
5.9021
0.22
6.8841
0.23
7.8662
0.24
8.8482
0.25
9.8303
0.26
10.8124
0.27
11.7944
0.28
12.7765
0.29
13.7585
0.3
14.7406
0.31
15.7226
0.32
16.7047
0.33
17.6867
0.34
18.6688
0.35000000000000003
19.6508
0.36
20.6329
0.37
21.6149
0.38
22.597
0.39
23.579
0.4
24.5611
0.41000000000000003
25.5431
0.42
26.5252
0.43
27.5072
0.44
28.4893
0.45
29.4713
0.46
30.4534
0.47000000000000003
31.4355
0.48
32.4175
0.49
33.3996
0.5
34.3816


0.15 appears to be when the logit would meet the intersection line. We should use that as the new threshold. 

#### d. Interpret the estimated coefficient for the total loans & leases to total assets ratio (TotLns&Lses/Assets) in terms of the odds of being financially weak.

8.3713% of that ratio is added to the probability that a bank is finantially weak. 

### 4. Competitive Auctions on eBay.com. The file eBayAuctions.csv contains information on 1972 auctions transacted on eBay.com during May–June 2004. The goal is to use these data to build a model that will distinguish competitive auctions from noncompetitive ones. A competitive auction is defined as an auction with at least two bids placed on the item being auctioned. The data include variables that describe the item (auction category), the seller (his or her eBay rating), and the auction terms that the seller selected (auction duration, opening price, currency, day of week of auction close). In addition, we have the price at which the auction closed. The goal is to predict whether or not an auction of interest will be competitive.

In [121]:
ebay_df = pd.read_csv(DATA / 'eBayAuctions.csv')
print(ebay_df.shape)
ebay_df.head(2)

(1972, 8)


,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive?
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0


### Data preprocessing. Create dummy variables for the categorical predictors. These include Category (18 categories), Currency (USD, GBP, Euro), EndDay (Monday–Sunday), and Duration (1, 3, 5, 7, or 10 days).

In [122]:
print(ebay_df.dtypes)


Category         object
currency         object
sellerRating      int64
Duration          int64
endDay           object
ClosePrice      float64
OpenPrice       float64
Competitive?      int64
dtype: object


#### a. Create pivot tables for the mean of the binary outcome (Competitive?) as a function of the various categorical variables (use the original variables, not the dummies). Use the information in the tables to reduce the number of dummies that will be used in the model. For example, categories that appear most similar with respect to the distribution of competitive auctions could be combined.

In [124]:
tbl = pd.crosstab(ebay_df["Competitive?"], ebay_df["Category"])
tbl

Category,Antique/Art/Craft,Automotive,Books,Business/Industrial,Clothing/Accessories,Coins/Stamps,Collectibles,Computer,Electronics,EverythingElse,Health/Beauty,Home/Garden,Jewelry,Music/Movie/Game,Photography,Pottery/Glass,SportingGoods,Toys/Hobbies
Competitive?,,,,,,,,,,,,,,,,,,
0,77,115,27,6,59,26,101,12,11,13,53,35,52,160,2,13,34,110
1,100,63,27,12,60,11,138,24,44,4,11,67,30,243,11,7,90,124


In [125]:
tbl = pd.crosstab(ebay_df["Competitive?"], ebay_df["currency"])
tbl

currency,EUR,GBP,US
Competitive?,,,
0,239,46,621
1,294,101,671


In [126]:
tbl = pd.crosstab(ebay_df["Competitive?"], ebay_df["Duration"])
tbl

Duration,1,3,5,7,10
Competitive?,,,,,
0,11,117,146,494,138
1,12,96,320,473,165


In [127]:
tbl = pd.crosstab(ebay_df["Competitive?"], ebay_df["endDay"])
tbl

endDay,Fri,Mon,Sat,Sun,Thu,Tue,Wed
Competitive?,,,,,,,
0,153,179,201,174,80,80,39
1,134,369,150,164,122,91,36


##### After you create pivot tables, combine the following categories to reduce the number of dummy variables for logistic regression: Sun, Wed, Fri for "endDay". "Business/Industrial", "Computer", and "Home/Garden" for 'Category'. "Antique/Art/Craft" and 'Collectibles' for 'Category'. "Automotive" and 'Pottery/Glass' for 'Category'. "Books" and 'Clothing/Accessories' for 'Category'.

In [128]:
for i in range(0, ebay_df.shape[0]):
    if ebay_df["Category"][i] in ['Business/Industrial', 'Computer', 'Home/Garden']:
        ebay_df['Category'][i] = 'Business/Industrial/Computer/Home/Garden'
    elif ebay_df["Category"][i] in ['Antique/Art/Craft', 'Collectibles']:
        ebay_df['Category'][i] = 'Antique/Art/Craft/Collectibles'
    elif ebay_df["Category"][i] in ['Automotive', 'Pottery/Glass']:
        ebay_df['Category'][i] = 'Automotive/Pottery/Glass'
    elif ebay_df["Category"][i] in ['Books', 'Clothing/Accessories']:
        ebay_df['Category'][i] = 'Books/Clothing/Accessories'
    if ebay_df['endDay'][i] in ['Sun', 'Wed', 'Fri']:
        ebay_df['endDay'][i] = 'Sun/Wed/Fri'


C:\Users\tanve\AppData\Local\Temp\ipykernel_21420\215902849.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ebay_df['Category'][i] = 'Automotive/Pottery/Glass'
C:\Users\tanve\AppData\Local\Temp\ipykernel_21420\215902849.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ebay_df['Category'][i] = 'Automotive/Pottery/Glass'
C:\Users\tanve\AppData\Local\Temp\ipykernel_21420\215902849.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [129]:
catagorize = ['Category', 'currency', 'endDay', 'Duration']
outcome = "Competitive?"
subset = catagorize.__add__([outcome])

In [130]:
for i in catagorize: 
    ebay_df[i] = ebay_df[i].astype('category')

print(ebay_df.dtypes)
print('\n') 
mod_ebay_df = pd.get_dummies(ebay_df,prefix_sep='_', drop_first=False)
print(mod_ebay_df.dtypes)
print(mod_ebay_df.shape)

Category        category
currency        category
sellerRating       int64
Duration        category
endDay          category
ClosePrice       float64
OpenPrice        float64
Competitive?       int64
dtype: object


sellerRating                                           int64
ClosePrice                                           float64
OpenPrice                                            float64
Competitive?                                           int64
Category_Antique/Art/Craft/Collectibles                 bool
Category_Automotive/Pottery/Glass                       bool
Category_Books/Clothing/Accessories                     bool
Category_Business/Industrial/Computer/Home/Garden       bool
Category_Coins/Stamps                                   bool
Category_Electronics                                    bool
Category_EverythingElse                                 bool
Category_Health/Beauty                                  bool
Category_Jewelry                                    

#### b. Split the data into training (60%) and validation (40%) datasets. Run a logistic model with all predictors with a cutoff of 0.5.

In [131]:
X = mod_ebay_df.drop(columns=outcome)
y = mod_ebay_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X,y, train_size=0.6, random_state=1)

In [132]:
logit_reg_predictor = LogisticRegression(penalty="l2", solver='lbfgs', C=1e24, multi_class='multinomial')
logit_reg_predictor.fit(train_X, train_y)

c:\Program Files\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1e+24, multi_class='multinomial')

In [133]:
predictions = logit_reg_predictor.predict(valid_X)
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
predictions_nominal[0:5]

[0, 1, 1, 1, 0]

In [134]:
print('intercept ', logit_reg_predictor.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg_predictor.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_reg_predictor.predict(valid_X), df = len(train_X.columns) + 1))

intercept  -0.07976927005718845
       sellerRating  ClosePrice  OpenPrice  \
coeff     -0.000017    0.045095  -0.051131   

       Category_Antique/Art/Craft/Collectibles  \
coeff                                 0.194535   

       Category_Automotive/Pottery/Glass  Category_Books/Clothing/Accessories  \
coeff                          -0.115574                            -0.236211   

       Category_Business/Industrial/Computer/Home/Garden  \
coeff                                           0.118196   

       Category_Coins/Stamps  Category_Electronics  Category_EverythingElse  \
coeff              -0.131249              0.052498                -0.064152   

       ...  Duration_1  Duration_3  Duration_5  Duration_7  Duration_10  \
coeff  ...   -0.011656   -0.125532    0.147446   -0.073767     -0.01626   

       endDay_Mon  endDay_Sat  endDay_Sun/Wed/Fri  endDay_Thu  endDay_Tue  
coeff    0.168965   -0.251586           -0.090723    0.124304   -0.030729  

[1 rows x 29 columns]

AIC 

In [135]:
classificationSummary(valid_y, logit_reg_predictor.predict(valid_X))

Confusion Matrix (Accuracy 0.7490)

       Prediction
Actual   0   1
     0 283  70
     1 128 308


#### c. If we want to predict at the start of an auction whether it will be competitive, we cannot use the information on the closing price. Run a logistic model with all predictors as above, excluding price. How does this model compare to the full model with respect to predictive accuracy?

In [136]:
mod_ebay_df = mod_ebay_df.drop(columns='ClosePrice')
X = mod_ebay_df.drop(columns=outcome)
y = mod_ebay_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X,y, train_size=0.6, random_state=1)

In [137]:
logit_reg_predictor = LogisticRegression(penalty="l2", solver='lbfgs', C=1e24, multi_class='multinomial')
logit_reg_predictor.fit(train_X, train_y)

LogisticRegression(C=1e+24, multi_class='multinomial')

In [138]:
print('intercept ', logit_reg_predictor.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg_predictor.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_reg_predictor.predict(valid_X), df = len(train_X.columns) + 1))

intercept  0.11716862147350404
       sellerRating  OpenPrice  Category_Antique/Art/Craft/Collectibles  \
coeff     -0.000016  -0.001615                                 0.275998   

       Category_Automotive/Pottery/Glass  Category_Books/Clothing/Accessories  \
coeff                          -0.285038                            -0.139455   

       Category_Business/Industrial/Computer/Home/Garden  \
coeff                                           0.305048   

       Category_Coins/Stamps  Category_Electronics  Category_EverythingElse  \
coeff              -0.213378              0.165642                -0.078644   

       Category_Health/Beauty  ...  Duration_1  Duration_3  Duration_5  \
coeff               -0.421014  ...    0.026687   -0.063426     0.25179   

       Duration_7  Duration_10  endDay_Mon  endDay_Sat  endDay_Sun/Wed/Fri  \
coeff   -0.063177    -0.034705    0.293893    -0.22673           -0.083213   

       endDay_Thu  endDay_Tue  
coeff    0.171418     -0.0382  

[1 r

In [139]:
classificationSummary(valid_y, logit_reg_predictor.predict(valid_X))

Confusion Matrix (Accuracy 0.6147)

       Prediction
Actual   0   1
     0 197 156
     1 148 288


The model with out a ending price has worse predictive power and a higher AIC score than the full model. It fits the test data worse than the model excluding the closing cost. 

#### d. Interpret the meaning of the coefficient for closing price. Does closing price have a practical significance? Is it statistically significant for predicting competitiveness of auctions? (Use a 10% significance level.)

~5% of the closing cost is added to the probability that the auction was competitive. I would say it can have practical significance, if the price grows to 111% of the starting price then the closing price should meet the 10% signifiacne level that is needed. 